In [165]:
# 运行环境： python-3.6.12 tensorflow-1.14.0  
import numpy as np
import pandas as pd
import tensorflow as tf
from geopy.distance import geodesic
import scipy.stats
import math
import os
from sklearn.preprocessing import StandardScaler #正态分布归一化
import warnings; warnings.simplefilter('ignore')

In [166]:
SIP_E = np.load('SIP_Time5_Weather10-Interval30-201701_03.npy')
SIP_30min_flow = np.load('SIP_30m_Flow.npy')


#地区数量Node
location_degree = SIP_30min_flow.shape[0]

#设置一个batch_size大小以及总共有多少块
batch_size = 1
batch_number = np.int(4320/18/batch_size)
one_epoch_size = np.int((4320-17)/batch_size)
#M
w3_degree = 4
#
M5_degree = 8
#GNN
w6_degree = 64

#将12个按多少分一组
GNN_degree = 3
number_of_GNN = np.int(12/3)
#LSTM_output_seq
output_seq_len = 6
#
num_stacked_layers = 1
hidden_dim_1 = 96

In [167]:
SIP_30min_flow_mean = np.mean(SIP_30min_flow,axis=1)
var = np.std(SIP_30min_flow,axis=1)
var.shape
SIP_30min_flow_mean = np.expand_dims(SIP_30min_flow_mean,1)
var = np.expand_dims(var,1)
SIP_30min_flow_mean = np.tile(SIP_30min_flow_mean,(108,4320))

In [168]:
SIP_30min_flow_mean = np.mean(SIP_30min_flow,axis=1)
SIP_30min_flow_mean = np.expand_dims(SIP_30min_flow_mean,1)

SIP_30min_flow_mean = np.tile(SIP_30min_flow_mean,(1,4320))
#var = np.tile(var,(1,4320))
SIP_30min_flow_mean.shape,var.shape

((108, 4320), (108, 1))

In [169]:
np.save('data/SIP_30m_mean.npy',SIP_30min_flow_mean)
np.save('data/SIP_30m_var.npy',var)

In [170]:
# normalization 
SIP_30min_flow_norm = (SIP_30min_flow - SIP_30min_flow_mean)/var
# reverse
SIP_30min_flow_rev = SIP_30min_flow_norm * var + SIP_30min_flow_mean

In [171]:
# 取外部变量x-y个时间段的平均值
def extract_x_sample_e(matrix, x, y):
    return np.mean(matrix[x:y, :], axis=0)


# 返回从x处截断的两个三维矩阵
def cut_matrix(matrix, x):
    return matrix[:,:,0:x], matrix[:,:,x:]


# 取矩阵
def get_target_matrix(matrix, x, y):
    return matrix[:, x:y]


 
    

In [172]:
#取batch组数据 返回训练数据batch*1*15  观测数据batch*108*6  每18个interval一个sample
def get_batch_data(batch_size,number_of_batch):
    batch_input1_data = np.zeros((batch_size,1,15))
    batch_input2_data = np.zeros((batch_size,108,12))
    batch_real_data = np.zeros((batch_size,108,6))
  #  tt = np.random.choice(range(14*48,4320), batch_size)
    
    #normalization_data = np.zeros((108,18))
    for i in range(batch_size):
        #j = np.random.randint(0,240)
        
        #batch_input_data[i] = np.expand_dims(extract_x_sample_e(SIP_E,number_of_batch*batch_size+i ,number_of_batch*batch_size+i+12 ),axis=0)
        #batch_real_data[i] = get_target_matrix(SIP_30min_flow, number_of_batch*batch_size+i+12 ,number_of_batch*batch_size+i+18)
        
        batch_input1_data[i] = np.expand_dims(extract_x_sample_e(SIP_E,number_of_batch*batch_size*18+i*18 ,number_of_batch*batch_size*18+i*18+12 ),axis=0)
        batch_input2_data[i] = get_target_matrix(SIP_30min_flow,number_of_batch*batch_size*18+i*18,number_of_batch*batch_size*18+i*18+12)
        # batch_real_data[i] = get_target_matrix(SIP_30min_flow, number_of_batch*batch_size*18+i*18+12 ,number_of_batch*batch_size*18+i*18+18)
        batch_real_data[i] = get_target_matrix(SIP_30min_flow, number_of_batch*batch_size*18+i*18+12 ,number_of_batch*batch_size*18+i*18+18)
        
        #batch_input_data[i] = np.expand_dims(extract_x_sample_e(SIP_E,j*18 ,j*18+12 ),axis=0)
        #batch_real_data[i] = get_target_matrix(SIP_30min_flow, j*18+12 ,j*18+18)
        
        
    return batch_input1_data,batch_input2_data,batch_real_data.transpose(0,2,1)
a,b,c = get_batch_data(1,4)


In [173]:
location_matrix = np.eye(location_degree,dtype=np.float32)
location_matrix = np.expand_dims(location_matrix,axis=1)

In [174]:
def build_graph(feed_previous = False,reuse_variables=False):
    tf.reset_default_graph()
    global_step = tf.Variable(
                  initial_value=0,
                  name="global_step",
                  trainable=False,
                  collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])

    def get_shape_sim_tf(x,y):
        x_diff = x - tf.roll(x, shift=1, axis=0)
        y_diff = y - tf.roll(y, shift=1, axis=0)
        x_ycos = (-tf.reduce_sum(tf.multiply(x,y))/(tf.sqrt(tf.reduce_sum(tf.multiply(x,x))+0.01)*tf.sqrt(tf.reduce_sum(tf.multiply(y,y))))+0.01)
        #x_ydist = tf.sqrt(tf.reduce_sum(tf.pow(x_diff-y_diff, 2)))

        return x_ycos

    #
    x1 = tf.placeholder(tf.float32,[batch_size,1,15])#时间天气向量 batch_size*1*15
    x2 = tf.placeholder(tf.float32,[batch_size,108,12])#流量
    #ys = tf.placeholder(tf.float32,[batch_size,location_degree,6])#batch_size*108*6
    Output_Seq = tf.placeholder(shape=(1, output_seq_len, location_degree),dtype=tf.float32)
    
    #FC w,b
  #  w0 = tf.Variable(tf.random_normal([location_degree,location_degree,M5_degree]),dtype=tf.float32,name='w0')
    #  w0 = tf.Variable(tf.random_normal([location_degree,location_degree,M5_degree]),dtype=tf.float32,name='w0')
    w0 = tf.Variable(tf.random_normal([location_degree,M5_degree]),dtype=tf.float32,name='w0')
    
    w1 = tf.Variable(tf.random_normal([M5_degree,1]),dtype=tf.float32,name='w1') # √


    #b1 = tf.Variable(tf.truncated_normal([location_degree,location_degree],stddev=0.1),dtype=tf.float32,name='b1')

    b2 = tf.Variable(tf.truncated_normal([1,5],stddev=0.1),dtype=tf.float32,name='b2')

    w2 = tf.Variable(tf.truncated_normal([5,1]),dtype=tf.float32,name='w2')#5*1    # √

    b3 = tf.Variable(tf.truncated_normal([1,10],stddev=0.1),dtype=tf.float32,name='b3')

    w3 = tf.Variable(tf.truncated_normal([10,w3_degree]),dtype=tf.float32,name='w3')#10*M

    w4 = tf.Variable(tf.truncated_normal([w3_degree,w3_degree]),dtype=tf.float32,name='w4')#M*6
 
    
    w4_1 = tf.Variable(tf.truncated_normal([w3_degree,output_seq_len]),dtype=tf.float32,name='w4_1')#M*6
    w4_2 = tf.Variable(tf.truncated_normal([output_seq_len,output_seq_len]),dtype=tf.float32,name='w4_2')#M*6
    f6_out = tf.Variable(tf.random_normal([batch_size,location_degree,w3_degree]),dtype=tf.float32,name='context')

    location_w_matrix = tf.Variable(tf.truncated_normal([location_degree,1,M5_degree]),dtype=tf.float32,name='M5')

    def add_layer(inputs,activation_function=None):
        #获得时间天气
        inputs = tf.cast(inputs,tf.float32)
        time_vector ,weather_vector = cut_matrix(inputs, 5) #b*1*5 b*1*10

        #(L+b1)*w0*w1
#         for i in range(location_degree):
#             tf.assign(location_w_matrix[i],tf.matmul(location_matrix[i],w0[i]))
      
        f1_out = tf.matmul(location_matrix, w0)
        f1_out = tf.squeeze(location_w_matrix,axis=1)#108*M
        f2_out = tf.matmul(f1_out, w1)

        f3_out = tf.map_fn(lambda x: tf.matmul(f2_out, x), time_vector) #b*108*5

        f4_out = tf.map_fn(lambda x: tf.matmul(x,w2), f3_out)#b*108*1

        f5_out = tf.matmul(f4_out,weather_vector)#b*108*10

        wb_mean, wb_var = tf.nn.moments(f5_out, [0,1,2])
        scale = tf.Variable(tf.ones([1]))
    
        offset = tf.Variable(tf.zeros([1]))
    
        variance_epsilon = 0.001
        f5_out = tf.nn.batch_normalization(f5_out, wb_mean, wb_var, offset, scale, variance_epsilon)
        
        f6_out = tf.map_fn(lambda x: tf.matmul(x,w3), f5_out)#b*108*M
        
        f6_out = tf.map_fn(lambda x: tf.matmul(x,w4), f6_out)#b*108*M
                
        Pred_Context_out = tf.map_fn(lambda x: tf.matmul(x,w4_1), f6_out) #b*108*6
        
        Pred_Context_out = tf.map_fn(lambda x: tf.matmul(x,w4_2), Pred_Context_out) #b*108*6
        
        if activation_function is None:
            outputs = f6_out
        else:
            outputs = activation_function(f6_out)
        return outputs, Pred_Context_out, f6_out
    
    FC_layer_1, Pred_Context_out,f6_out = add_layer(x1,activation_function=tf.nn.relu)
    
    Pred_Context_out = tf.transpose(Pred_Context_out,[0,2,1])
    print("Pred_contextshape: ",Pred_Context_out.shape)
    

    
    #FC——得到b*108*108的参数
    w5 = tf.Variable(tf.truncated_normal([w3_degree,location_degree]),dtype=tf.float32,name='w5')
    b5 = tf.Variable(tf.truncated_normal([location_degree,w3_degree]),dtype=tf.float32,name='b5')
    #Gnn参数
    #108*3的b
    b6 = tf.Variable(tf.truncated_normal([batch_size,location_degree,GNN_degree]),dtype=tf.float32,name='b6')
    w6 = tf.Variable(tf.truncated_normal([GNN_degree, 64]),dtype=tf.float32,name='w6')
    w7 = tf.Variable(tf.truncated_normal([64,8]),dtype=tf.float32,name='w7')
    w8 = tf.Variable(tf.truncated_normal([8,1]),dtype=tf.float32,name='w8')
    
   # w9 = tf.Variable(tf.truncated_normal([3,1]),dtype=tf.float32,name='w9')
  #  w10 = tf.Variable(tf.truncated_normal([3,1]),dtype=tf.float32,name='w10')
  #  w11 = tf.Variable(tf.truncated_normal([3,1]),dtype=tf.float32,name='w11')
    
    #得到batch个108*108个context矩阵
    def get_ADJ(inputs,activation_function=None):
        inputs = tf.cast(inputs,tf.float32)
        f1 = tf.matmul(inputs,w5)

        if activation_function is None:
            outputs = f1
        else:
            outputs = activation_function(f1)
        return outputs
    
    #得到context
    ADJ_matrix = get_ADJ(FC_layer_1,activation_function=tf.nn.leaky_relu) 
    
    
    def Long_termGCN(input1,  Adj_M):
    #输入为b*108*3  b*108*108
        #第一层
        #得到b*108*3
        layer1_temp = tf.matmul(Adj_M, input1)
        #print(layer1_temp.shape)
        #b*108*3   *    3*M   =  b*108*M
        layer_1_output = tf.map_fn(lambda x: tf.matmul(x, w6), layer1_temp)
        #b*108*M   *    M*1   =  b*108*1
        #layer_1_output = tf.map_fn(lambda x: tf.matmul(x, w7), layer_1_output)
        
        wb_mean, wb_var = tf.nn.moments(layer_1_output, [0,1,2])
        scale = tf.Variable(tf.ones([1]))
    
        offset = tf.Variable(tf.zeros([1])) 
    
        variance_epsilon = 0.001
        layer_1_output = tf.nn.batch_normalization(layer_1_output, wb_mean, wb_var, offset, scale, variance_epsilon)
        
       
        
        layer_1_output = tf.map_fn(lambda x: tf.matmul(x, w7), layer_1_output)
     #   layer_1_output = tf.matmul(Adj_M, layer_1_output)
        
        layer_1_output = tf.map_fn(lambda x: tf.matmul(x, w8), layer_1_output)
        
     #   layer_1_output = tf.map_fn(lambda x: tf.matmul(x, w9), layer_1_output)
    #    layer_1_output = tf.map_fn(lambda x: tf.matmul(x, w10), layer_1_output)
    #    layer_1_output = tf.map_fn(lambda x: tf.matmul(x, w11), layer_1_output)
        return layer_1_output
    
    
    def lstm(Output6,Output_Seq,num_stacked_layers,input_size, hidden_dim,feed_previous): 
        
        weightslstm={
         'in':tf.Variable(tf.truncated_normal(shape=[input_size,hidden_dim], stddev=0.1)), 
         'out':tf.Variable(tf.truncated_normal(shape=[hidden_dim,input_size], stddev=0.1)) 
        }
        biaseslstm={
                'in':tf.Variable(tf.truncated_normal(shape=[hidden_dim,], stddev=0.1)), 
                'out':tf.Variable(tf.truncated_normal(shape=[input_size,], stddev=0.1)) 
               }

        
        ## Seq2Seq Parameters
        ## Parameters
        enc_inp = Output6

        print("len_enc_inp:",len(enc_inp))
        print(enc_inp)
        # Decoder: target outputs
        target_seq = [
            tf.placeholder(tf.float32, shape=(1, input_size), name="y".format(t))
              for t in range(output_seq_len)
        ]

        target_seq = Output_Seq
        print("len_target_seq:",len(target_seq))
        dec_inp = [ tf.zeros_like(target_seq[0], dtype=tf.float32, name="GO") ] + target_seq[:-1]
    
    
        cells = []
        for i in range(num_stacked_layers):
            with tf.variable_scope('RNN_{}'.format(i)):
                cells.append(tf.contrib.rnn.LSTMCell(num_units = hidden_dim, activation = tf.nn.relu)) # activation = tf.nn.leaky_relu

        cell = tf.contrib.rnn.MultiRNNCell(cells)
        def _rnn_decoder(decoder_inputs,
                        initial_state,
                        cell,
                        loop_function=None,
                        scope=None):
            
            state = initial_state
            outputs = []
            prev = None
            for i, inp in enumerate(decoder_inputs):
                if loop_function is not None and prev is not None:
                    with tf.variable_scope("loop_function", reuse=tf.AUTO_REUSE):
                        inp = loop_function(prev, i)
                if i > 0:
                    tf.get_variable_scope().reuse_variables()
                output, state = cell(inp, state)
                outputs.append(output)
                if loop_function is not None:
                    prev = output
            return outputs, state


        def _basic_rnn_seq2seq(encoder_inputs,
                              decoder_inputs,
                              cell,
                              feed_previous,
                              dtype=tf.float32,
                              scope=None):
        #  """Basic RNN sequence-to-sequence model.
        #  """
        # Encoder  
        # 运行 encoder,把待编码的输入RNN中得到enc_state
            enc_cell = cell
            _, enc_state = tf.contrib.rnn.static_rnn(enc_cell, encoder_inputs, dtype=dtype)
            # Decoder 
            #如何feed_previous = True 的话 就把先前decoder 输出的放到下一个时间步去进行decoder
            if feed_previous:
                return _rnn_decoder(decoder_inputs, enc_state, cell, _loop_function)
            else:
                return _rnn_decoder(decoder_inputs, enc_state, cell)
        #print(tf.get_variable_scope().reuse)
        def _loop_function(prev, _):
          '''Naive implementation of loop function for _rnn_decoder. Transform prev from 
          dimension [batch_size x hidden_dim] to [batch_size x output_dim], which will be
          used as decoder input of next time step '''
          return tf.matmul(prev, weightslstm['out']) + biaseslstm['out']
        
        dec_outputs, dec_memory = _basic_rnn_seq2seq(
            enc_inp, 
            dec_inp,  ##输入带有context的数据
            cell, 
            feed_previous = feed_previous
        )
        # list:内部是6个1*108tensor张量
        Pred = [tf.nn.relu(tf.matmul(i, weightslstm['out'])+biaseslstm['out']) for i in dec_outputs]
        return Pred
    
    Input_Seq_F = []
    #number_of_GNN = 4
    for i in range(number_of_GNN):
        #print(Long_termGCN(x2[:,:,i*3:i*3+3], ADJ_matrix).shape)
        output = Long_termGCN(x2[:,:,i*3:i*3+3], ADJ_matrix)
        output = tf.squeeze(output,axis=2)
        Input_Seq_F.append(output)  
    #print(Input_Seq_F)
    #4 * 108 * 1
    Output_Seq_F = []  # 6*108
    for i in range(6):
        Output_Seq_F.append(Output_Seq[:,i ,:])
    #print("output##############################",Output_Seq_F)

    #LSTM
    with tf.variable_scope("Pred_RNN"):  
            Pred_Result = lstm(Input_Seq_F,Output_Seq_F,num_stacked_layers,108,hidden_dim_1,False)
    loss1 = 0
    loss2 = 0
    loss_shape = 0
    for _y, _Y in zip(Output_Seq_F, Pred_Result):
        print(_y.shape, _Y.shape)
        loss1 += tf.reduce_mean(tf.keras.losses.mse(_y ,_Y))
        loss2 += tf.reduce_mean(tf.keras.losses.mape(_y ,_Y))
    
    Pred_Result = tf.reshape(Pred_Result,[1,output_seq_len, location_degree])
    Output_Seq1 = tf.reshape(Output_Seq,[1,output_seq_len, location_degree])
    print("Pred_rsut:",Pred_Result.shape)
    for i in range(location_degree):
       # loss_shape += get_shape_sim_tf(Pred_Context_out[0,:,i],Output_Seq1[0,:,i]) +  get_shape_sim_tf(Pred_Result[0,:,i],Output_Seq1[0,:,i])
       loss_shape += get_shape_sim_tf(Pred_Result[0,:,i],Output_Seq1[0,:,i])
       #loss_shape += get_shape_sim_tf(Pred_Context_out[0,:,i],Output_Seq1[0,:,i]) 
    #print("#########",loss1)
    loss_FC = tf.reduce_mean(tf.keras.losses.mape(Output_Seq,Pred_Context_out))
    print("pred_shape:", Pred_Context_out.shape)
   
    loss2 = loss2/6
    loss_FC = loss_FC/2
   
    loss_sum =loss2  +  loss_FC  + 0.3 * loss_shape
    #print("#########",loss1)
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(0.0001,global_step,15,decay_rate=0.98,staircase=True)  
    train = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_sum)#学习率下降
    saver = tf.train.Saver
    context_embed = f6_out 
    return dict(
        x1 = x1,
        x2 = x2,
        Pred_Context_out = Pred_Context_out,
        Pred_Result = Pred_Result, # √  Prediction
        Output_Seq = Output_Seq, # √ Ground truth
        context_embed = context_embed, # √ Context
        loss_FC = loss_FC,
        loss1 = loss1,
        loss2 = loss2,   # √  
        loss_shape = loss_shape, 
        train = train,      
        saver = saver,
        ADJ_matrix = ADJ_matrix # √
        )

In [175]:
graph = build_graph(feed_previous = False,reuse_variables=False)

Pred_contextshape:  (1, 6, 108)
len_enc_inp: 4
[<tf.Tensor 'Squeeze_1:0' shape=(1, 108) dtype=float32>, <tf.Tensor 'Squeeze_2:0' shape=(1, 108) dtype=float32>, <tf.Tensor 'Squeeze_3:0' shape=(1, 108) dtype=float32>, <tf.Tensor 'Squeeze_4:0' shape=(1, 108) dtype=float32>]
len_target_seq: 6
(1, 108) (1, 108)
(1, 108) (1, 108)
(1, 108) (1, 108)
(1, 108) (1, 108)
(1, 108) (1, 108)
(1, 108) (1, 108)
Pred_rsut: (1, 6, 108)
pred_shape: (1, 6, 108)


In [176]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
sess = tf.Session()
sess.run(init)

In [177]:
Pred_ContextL = [] 
Pred_ResultL= []
Output_Seq_L = []
Context_embedL = []
        
for epoch in range(500):
    min_mape = 50
    sum_mpape = 0
    for i in range(batch_number):
        input1,input2,real_label = get_batch_data(batch_size,i)
        #print(input1,input2)
        feed_dict={graph['x1']:input1,graph['x2']:input2,graph['Output_Seq']:real_label}
        _,Pred_Context_out,Pred_Result,loss2,lossFC,ADJ_matrix,Output_seq,loss_shape = sess.run([graph['train'],graph['Pred_Context_out'],graph['Pred_Result'],graph['loss2'],graph['loss_FC'],graph['ADJ_matrix'],graph['Output_Seq'],graph['loss_shape']],feed_dict)
        sum_mpape = sum_mpape + loss2
        #_,loss1,Pred_Context_out,Pred_Result,Output_Seq,context_embed, loss2,lossFC,ADJ_matrix= sess.run([graph['train'],graph['loss1'],graph['Pred_Context_out'],graph['Pred_Result'],graph['Output_Seq'],graph['context_embed'],graph['loss2'],graph['loss_FC'],graph['ADJ_matrix']],feed_dict)
        if(i%240==0):
            #print(  "MSE:",loss1,"  MAPE",loss2,"预测结果:",Pred_Result[0]," 目标: ",real_label[:,0,:])
            #print( "Epoch:", epoch,  "  MAPE",loss2,"Conxtloss", lossFC,"loss_shape:", loss_shape," Pred_result:", Pred_Result[0][:,15],Output_seq[0][:,15])
            print( "Epoch:", epoch,  "  MAPE",loss2)
    sum_mpape = sum_mpape/batch_number
    print("sum_mpape: ",sum_mpape)

Epoch: 0   MAPE 90.30061
sum_mpape:  79.67108960151673
Epoch: 1   MAPE 76.93974
sum_mpape:  64.905561876297
Epoch: 2   MAPE 75.5257
sum_mpape:  63.579222679138184
Epoch: 3   MAPE 74.11926
sum_mpape:  63.24172336260478
Epoch: 4   MAPE 74.16232
sum_mpape:  63.08619360923767
Epoch: 5   MAPE 73.39907
sum_mpape:  62.66040563583374
Epoch: 6   MAPE 73.08414
sum_mpape:  62.61097442309062
Epoch: 7   MAPE 72.68962
sum_mpape:  62.300768502553304
Epoch: 8   MAPE 72.337746
sum_mpape:  61.79799281756083
Epoch: 9   MAPE 71.684586
sum_mpape:  61.453480911254886
Epoch: 10   MAPE 70.99833
sum_mpape:  60.90745412508647
Epoch: 11   MAPE 70.32652
sum_mpape:  60.6958615620931
Epoch: 12   MAPE 69.61961
sum_mpape:  60.309106254577635
Epoch: 13   MAPE 68.92993
sum_mpape:  60.08208301862081
Epoch: 14   MAPE 67.99284
sum_mpape:  59.61430368423462
Epoch: 15   MAPE 67.19273
sum_mpape:  59.15302845637004
Epoch: 16   MAPE 66.21699
sum_mpape:  58.57808105150858
Epoch: 17   MAPE 65.54939
sum_mpape:  58.10820333162943


sum_mpape:  38.721429785092674
Epoch: 145   MAPE 50.27945
sum_mpape:  38.462616388003035
Epoch: 146   MAPE 49.9763
sum_mpape:  38.7913730541865
Epoch: 147   MAPE 48.71179
sum_mpape:  38.825009950002034
Epoch: 148   MAPE 49.57739
sum_mpape:  38.66506128311157
Epoch: 149   MAPE 49.42627
sum_mpape:  38.65980831782023
Epoch: 150   MAPE 49.1286
sum_mpape:  38.7143514474233
Epoch: 151   MAPE 50.6591
sum_mpape:  39.044642837842304
Epoch: 152   MAPE 50.746075
sum_mpape:  38.59942820072174
Epoch: 153   MAPE 49.7922
sum_mpape:  38.4282204469045
Epoch: 154   MAPE 50.058586
sum_mpape:  38.57911269664764
Epoch: 155   MAPE 50.823845
sum_mpape:  38.37860809167226
Epoch: 156   MAPE 50.973705
sum_mpape:  38.43101101716359
Epoch: 157   MAPE 50.173504
sum_mpape:  38.30929468472799
Epoch: 158   MAPE 50.22427
sum_mpape:  38.33499173323313
Epoch: 159   MAPE 49.482037
sum_mpape:  38.11833102703095
Epoch: 160   MAPE 50.563904
sum_mpape:  38.184517065684
Epoch: 161   MAPE 49.9131
sum_mpape:  38.50777295430501


sum_mpape:  29.686501185099285
Epoch: 287   MAPE 33.112274
sum_mpape:  30.123866526285806
Epoch: 288   MAPE 32.46126
sum_mpape:  29.698764578501383
Epoch: 289   MAPE 32.235992
sum_mpape:  30.06750196615855
Epoch: 290   MAPE 31.328651
sum_mpape:  31.208383146921793
Epoch: 291   MAPE 30.280796
sum_mpape:  30.534521063168842
Epoch: 292   MAPE 30.779533
sum_mpape:  30.010034489631654
Epoch: 293   MAPE 31.011856
sum_mpape:  29.726289335886637
Epoch: 294   MAPE 31.146225
sum_mpape:  29.77938459714254
Epoch: 295   MAPE 30.765457
sum_mpape:  31.22397372722626
Epoch: 296   MAPE 29.99982
sum_mpape:  29.712166770299277
Epoch: 297   MAPE 30.301147
sum_mpape:  29.28126360575358
Epoch: 298   MAPE 30.941761
sum_mpape:  29.242045704523722
Epoch: 299   MAPE 30.91223
sum_mpape:  29.16640216509501
Epoch: 300   MAPE 31.37792
sum_mpape:  28.972971693674722
Epoch: 301   MAPE 31.45287
sum_mpape:  29.091223057111105
Epoch: 302   MAPE 31.430904
sum_mpape:  28.887386695543924
Epoch: 303   MAPE 31.117874
sum_mpa

sum_mpape:  26.453866291046143
Epoch: 428   MAPE 27.765137
sum_mpape:  26.47576138575872
Epoch: 429   MAPE 27.4632
sum_mpape:  26.372462836901345
Epoch: 430   MAPE 27.921432
sum_mpape:  26.341869763533275
Epoch: 431   MAPE 27.763817
sum_mpape:  26.30457765261332
Epoch: 432   MAPE 27.514957
sum_mpape:  26.302937920888265
Epoch: 433   MAPE 27.646078
sum_mpape:  26.25602417786916
Epoch: 434   MAPE 27.526617
sum_mpape:  26.16420431534449
Epoch: 435   MAPE 27.447834
sum_mpape:  26.200762518246968
Epoch: 436   MAPE 27.540289
sum_mpape:  26.228417122364043
Epoch: 437   MAPE 27.388988
sum_mpape:  26.217111444473268
Epoch: 438   MAPE 27.404623
sum_mpape:  26.05965623060862
Epoch: 439   MAPE 27.343006
sum_mpape:  26.10418742497762
Epoch: 440   MAPE 27.523977
sum_mpape:  26.21851682662964
Epoch: 441   MAPE 27.238937
sum_mpape:  26.019590202967326
Epoch: 442   MAPE 27.552626
sum_mpape:  26.04269288778305
Epoch: 443   MAPE 27.374058
sum_mpape:  26.048266665140787
Epoch: 444   MAPE 27.573948
sum_mpa

In [184]:
for epoch in range(10):
    min_mape = 50
    sum_mpape = 0
    for i in range(batch_number):
        input1,input2,real_label = get_batch_data(batch_size,i)
        #print(input1,input2)
        feed_dict={graph['x1']:input1,graph['x2']:input2,graph['Output_Seq']:real_label}
        _,Pred_Context_out,Pred_Result,loss2,lossFC,ADJ_matrix,Output_seq,loss_shape = sess.run([graph['train'],graph['Pred_Context_out'],graph['Pred_Result'],graph['loss2'],graph['loss_FC'],graph['ADJ_matrix'],graph['Output_Seq'],graph['loss_shape']],feed_dict)
        sum_mpape = sum_mpape + loss2
        #_,loss1,Pred_Context_out,Pred_Result,Output_Seq,context_embed, loss2,lossFC,ADJ_matrix= sess.run([graph['train'],graph['loss1'],graph['Pred_Context_out'],graph['Pred_Result'],graph['Output_Seq'],graph['context_embed'],graph['loss2'],graph['loss_FC'],graph['ADJ_matrix']],feed_dict)
        if(i%240==0):
            #print(  "MSE:",loss1,"  MAPE",loss2,"预测结果:",Pred_Result[0]," 目标: ",real_label[:,0,:])
            #print( "Epoch:", epoch,  "  MAPE",loss2,"Conxtloss", lossFC,"loss_shape:", loss_shape," Pred_result:", Pred_Result[0][:,15],Output_seq[0][:,15])
            print( "Epoch:", epoch,  "  MAPE",loss2)
    sum_mpape = sum_mpape/batch_number
    print("sum_mpape: ",sum_mpape)

Epoch: 0   MAPE 24.471413
sum_mpape:  23.68597813049952
Epoch: 1   MAPE 24.318329
sum_mpape:  23.695165864626567
Epoch: 2   MAPE 24.3778
sum_mpape:  23.688615127404532
Epoch: 3   MAPE 24.329456
sum_mpape:  24.166069356600442
Epoch: 4   MAPE 25.134335
sum_mpape:  23.760733763376873
Epoch: 5   MAPE 24.404121
sum_mpape:  24.42713010708491
Epoch: 6   MAPE 24.861294
sum_mpape:  24.252546771367392
Epoch: 7   MAPE 24.462543
sum_mpape:  24.17449346780777
Epoch: 8   MAPE 24.344397
sum_mpape:  24.179450885454813
Epoch: 9   MAPE 24.632973
sum_mpape:  23.921228619416556


In [179]:
#保存最后一次训练的数据，可以用作评估(不要改动batch_size),都存为240*108*6
#存取FC的预测 
output_data1 = np.ones((batch_number*batch_size,108,6))
#存取GNN的预测
output_data2 = np.ones((batch_number*batch_size,108,6))
#存取对应的真实值
real_data = np.ones((batch_number*batch_size,108,6))

for i in range(batch_number):
        input1,input2,real_label = get_batch_data(batch_size,i)
        
        #训练并得到相应的预测和loss
        feed_dict={graph['x1']:input1,graph['x2']:input2,graph['Output_Seq']:real_label}
        _,loss1,loss2,loss_FC,Pred_Result,Pred_Context_out,context_embed,ADJ_matrix = sess.run([graph['train'],graph['loss1'],graph['loss2'],graph['loss_FC'],graph['Pred_Result'],graph['Pred_Context_out'],graph['context_embed'],graph['ADJ_matrix']],feed_dict)
        
        #将每一个batch的数据存起来
        output_data1[i,:,:] = Pred_Context_out.transpose(0,2,1)[0]
        real_data[i] = real_label.transpose(0,2,1)[0]
        for j in range(6):
            output_data2[i,:,j] = Pred_Result[0][j,:]
        
        if(i%24==0):
            #print(  "MSE:",loss1,"  MAPE",loss2,"预测结果:",Pred_Result[0]," 目标: ",real_label[:,0,:])
            print(  "MSE:",loss1,"  MAPE",loss2, " FC_loss_mape:",loss_FC)
            print("预测结果:",Pred_Result[0][0,15:20],"\n 目标: ",real_label[0,0,15:20])
np.save('data/Context_output.npy',output_data1)
np.save('data/GNN_output.npy',output_data2)
np.save('data/GNN_realdata.npy',real_data)
np.save('data/GNN_context_embed.npy',context_embed.squeeze(axis=0))
np.save('data/GNN_context_matrix.npy',ADJ_matrix.squeeze(axis=0))

MSE: 30684.371   MAPE 24.571455  FC_loss_mape: 37.08713
预测结果: [63.109028 63.927647 83.38756  36.130096 70.47493 ] 
 目标:  [ 75.  79. 172.  74. 146.]
MSE: 176769.02   MAPE 29.89264  FC_loss_mape: 42.462154
预测结果: [ 89.402214  96.981606 118.19715   60.886074 100.18897 ] 
 目标:  [143. 117. 274. 118. 267.]
MSE: 104953.0   MAPE 27.184048  FC_loss_mape: 40.771107
预测结果: [104.51459   98.57275  128.17105   57.172256 112.6937  ] 
 目标:  [144. 107. 268. 113. 241.]
MSE: 7481.6494   MAPE 27.051487  FC_loss_mape: 29.35692
预测结果: [ 0.          0.99654114 59.46162    48.289196   48.82595   ] 
 目标:  [ 48.  49. 119.  47.  73.]
MSE: 343914.53   MAPE 24.107117  FC_loss_mape: 42.082794
预测结果: [113.353546 104.58995  137.08067   59.081673 123.09593 ] 
 目标:  [128. 113. 117.  49.  72.]
MSE: 186358.34   MAPE 28.918982  FC_loss_mape: 42.680553
预测结果: [117.26412  105.21964  133.8081    60.400738 121.99939 ] 
 目标:  [187. 151. 143.  68. 122.]
MSE: 162110.66   MAPE 25.643156  FC_loss_mape: 42.417477
预测结果: [114.544136 107.7

In [185]:
#测试
#初始batch
batch_start = 0
#结尾batch
batch_end = 240
#每隔多少取一个
batch_interval = 2
mape = 0
for i in range(batch_start,batch_end,batch_interval):
    input1,input2,real_label = get_batch_data(batch_size,i)
    #print(input1,input2)
    feed_dict={graph['x1']:input1,graph['x2']:input2,graph['Output_Seq']:real_label}
    _,loss1,loss2,loss_FC,Pred_Result,Pred_Context_out,context_embed,ADJ_matrix,Output_seq = sess.run([graph['train'],graph['loss1'],graph['loss2'],graph['loss_FC'],graph['Pred_Result'],graph['Pred_Context_out'],graph['context_embed'],graph['ADJ_matrix'],graph['Output_Seq']],feed_dict)
    print("\n")
    print("Batch:",i, "TestSample: {}---{}   ".format(i*18,i*18+18))
    print( "MAPE",loss2, "Conxtloss", lossFC)
    print("Pred_result:", Pred_Result[0][:,15],"\nReal_label",Output_seq[0][:,15])



Batch: 0 TestSample: 0---18   
MAPE 24.339611 Conxtloss 40.45152
Pred_result: [ 63.137836 126.57937  228.54805  313.63037  374.9515   435.60422 ] 
Real_label [ 75. 102. 180. 300. 396. 525.]


Batch: 2 TestSample: 36---54   
MAPE 30.992336 Conxtloss 40.45152
Pred_result: [145.89302  135.31557   95.15185   67.99739   58.353992  46.778698] 
Real_label [125. 116.  90.  82.  63.  42.]


Batch: 4 TestSample: 72---90   
MAPE 19.791958 Conxtloss 40.45152
Pred_result: [422.9172  679.8204  544.4015  586.4971  550.45654 550.23694] 
Real_label [692. 687. 615. 582. 560. 510.]


Batch: 6 TestSample: 108---126   
MAPE 16.676014 Conxtloss 40.45152
Pred_result: [603.7755  612.5392  584.1598  602.8427  619.202   614.47736] 
Real_label [554. 594. 512. 524. 515. 617.]


Batch: 8 TestSample: 144---162   
MAPE 28.44793 Conxtloss 40.45152
Pred_result: [ 92.26492 212.07832 476.79993 765.2735  646.8296  565.88275] 
Real_label [125. 376. 800. 540. 477. 598.]


Batch: 10 TestSample: 180---198   
MAPE 30.29424 


Batch: 104 TestSample: 1872---1890   
MAPE 23.471067 Conxtloss 40.45152
Pred_result: [ 107.32301  213.5696   433.843    872.6355  1256.6056  1395.7965 ] 
Real_label [ 160.  294.  670. 1293. 1312. 1357.]


Batch: 106 TestSample: 1908---1926   
MAPE 22.008507 Conxtloss 40.45152
Pred_result: [ 84.022766 108.928444  72.71252   58.476185  47.3502    37.610706] 
Real_label [180.  98.  83.  49.  45.  29.]


Batch: 108 TestSample: 1944---1962   
MAPE 20.885357 Conxtloss 40.45152
Pred_result: [ 452.1393  1269.6588  1028.5056   878.68964  739.25     714.23486] 
Real_label [1572. 1386. 1145.  895.  910.  956.]


Batch: 110 TestSample: 1980---1998   
MAPE 20.871439 Conxtloss 40.45152
Pred_result: [ 405.17682  838.91046  940.4353   968.0232  1010.26825 1038.8882 ] 
Real_label [1026. 1018. 1124. 1184. 1201. 1188.]


Batch: 112 TestSample: 2016---2034   
MAPE 27.886358 Conxtloss 40.45152
Pred_result: [ 92.49066 154.56633 278.00366 473.71243 673.41095 761.70966] 
Real_label [106. 188. 400. 658. 886. 

Batch: 206 TestSample: 3708---3726   
MAPE 18.948143 Conxtloss 40.45152
Pred_result: [498.1918  454.51886 443.69724 485.76245 451.93848 450.5404 ] 
Real_label [346. 321. 406. 434. 427. 478.]


Batch: 208 TestSample: 3744---3762   
MAPE 26.44957 Conxtloss 40.45152
Pred_result: [ 153.2942   225.52438  417.82037  726.73376 1011.4833  1007.431  ] 
Real_label [ 141.  205.  361. 1017. 1268. 1282.]


Batch: 210 TestSample: 3780---3798   
MAPE 25.227974 Conxtloss 40.45152
Pred_result: [88.4314   58.9283   45.634125 46.6516   40.566895 38.429356] 
Real_label [69. 61. 59. 54. 46. 37.]


Batch: 212 TestSample: 3816---3834   
MAPE 19.162888 Conxtloss 40.45152
Pred_result: [584.6714  423.48566 319.73993 288.69086 252.05455 244.47057] 
Real_label [407. 332. 293. 322. 299. 249.]


Batch: 214 TestSample: 3852---3870   
MAPE 13.895942 Conxtloss 40.45152
Pred_result: [503.55142 444.8172  488.3889  510.9224  551.86035 510.14438] 
Real_label [491. 558. 607. 632. 573. 622.]


Batch: 216 TestSample: 3888---

In [ ]:
for epoch in range(200):
    for i in range(batch_number):
        input1,input2,real_label = get_batch_data(batch_size,i)
        #print(input1,input2)
        feed_dict={graph['x1']:input1,graph['x2']:input2,graph['Output_Seq']:real_label}
        _,loss1,Pred_Result,loss2,lossFC,ADJ_matrix,Output_seq = sess.run([graph['train'],graph['Pred_Context_out'],graph['Pred_Result'],graph['loss2'],graph['loss_FC'],graph['ADJ_matrix'],graph['Output_Seq']],feed_dict)
            
        #_,loss1,Pred_Context_out,Pred_Result,Output_Seq,context_embed, loss2,lossFC,ADJ_matrix= sess.run([graph['train'],graph['loss1'],graph['Pred_Context_out'],graph['Pred_Result'],graph['Output_Seq'],graph['context_embed'],graph['loss2'],graph['loss_FC'],graph['ADJ_matrix']],feed_dict)
        if(i%240==0):
            #print(  "MSE:",loss1,"  MAPE",loss2,"预测结果:",Pred_Result[0]," 目标: ",real_label[:,0,:])
            print( "Epoch:", epoch, "  MAPE",loss2,"Conxtloss", lossFC,"Pred_result:", Pred_Result[0][:,15],Output_seq[0][:,15])

In [90]:
for epoch in range(300):
    for i in range(batch_number):
        input1,input2,real_label = get_batch_data(batch_size,i)
        #print(input1,input2)
        feed_dict={graph['x1']:input1,graph['x2']:input2,graph['Output_Seq']:real_label}
        _,loss1,Pred_Result,loss2,lossFC,ADJ_matrix,Output_seq = sess.run([graph['train'],graph['Pred_Context_out'],graph['Pred_Result'],graph['loss2'],graph['loss_FC'],graph['ADJ_matrix'],graph['Output_Seq']],feed_dict)
            
        #_,loss1,Pred_Context_out,Pred_Result,Output_Seq,context_embed, loss2,lossFC,ADJ_matrix= sess.run([graph['train'],graph['loss1'],graph['Pred_Context_out'],graph['Pred_Result'],graph['Output_Seq'],graph['context_embed'],graph['loss2'],graph['loss_FC'],graph['ADJ_matrix']],feed_dict)
        if(i%240==0):
            #print(  "MSE:",loss1,"  MAPE",loss2,"预测结果:",Pred_Result[0]," 目标: ",real_label[:,0,:])
            print( "Epoch:", epoch, "  MAPE",loss2,"Conxtloss", lossFC,"Pred_result:", Pred_Result[0][:,15],Output_seq[0][:,15])

Epoch: 0   MAPE 35.173004 Conxtloss 12.745356 Pred_result: [109.571045 107.88966  176.85127  256.62006  359.36816  482.67126 ] [ 75. 102. 180. 300. 396. 525.]
Epoch: 1   MAPE 35.305824 Conxtloss 12.746304 Pred_result: [113.30425 107.65812 176.00102 254.37044 355.85703 479.72302] [ 75. 102. 180. 300. 396. 525.]
Epoch: 2   MAPE 36.276558 Conxtloss 12.747824 Pred_result: [117.87327 100.24497 173.27757 253.40921 356.5561  481.73642] [ 75. 102. 180. 300. 396. 525.]
Epoch: 3   MAPE 34.939743 Conxtloss 12.748443 Pred_result: [111.61251 106.81079 176.02304 255.52875 358.1844  483.37518] [ 75. 102. 180. 300. 396. 525.]
Epoch: 4   MAPE 34.94538 Conxtloss 12.750341 Pred_result: [112.475136 108.1655   176.36792  255.28423  357.3441   482.09967 ] [ 75. 102. 180. 300. 396. 525.]
Epoch: 5   MAPE 34.71719 Conxtloss 12.7500515 Pred_result: [109.83979 106.90159 175.92001 255.51698 358.15787 483.5083 ] [ 75. 102. 180. 300. 396. 525.]
Epoch: 6   MAPE 35.087776 Conxtloss 12.748362 Pred_result: [111.04892  

In [99]:
#保存最后一次训练的数据，可以用作评估(不要改动batch_size),都存为240*108*6
#存取FC的预测 
output_data1 = np.ones((one_epoch_size*batch_size,108,6))
#存取GNN的预测
output_data2 = np.ones((one_epoch_size*batch_size,108,6))
#存取对应的真实值
real_data = np.ones((one_epoch_size*batch_size,108,6))

for i in range(batch_number):
        input1,input2,real_label = get_batch_data(batch_size,i)
        
        #训练并得到相应的预测和loss
        feed_dict={graph['x1']:input1,graph['x2']:input2,graph['Output_Seq']:real_label}
        _,loss1,loss2,loss_FC,Pred_Result,Pred_Context_out= sess.run([graph['train'],graph['loss1'],graph['loss2'],graph['loss_FC'],graph['Pred_Result'],graph['Pred_Context_out']],feed_dict)
        
        #将每一个batch的数据存起来
        output_data1[i,:,:] = Pred_Context_out.transpose(0,2,1)[0]
        real_data[i] = real_label.transpose(0,2,1)[0]
        for j in range(6):
            output_data2[i,:,j] = Pred_Result[0][j,:]
        
        if(i%24==0):
            #print(  "MSE:",loss1,"  MAPE",loss2,"预测结果:",Pred_Result[0]," 目标: ",real_label[:,0,:])
            print(  "MSE:",loss1,"  MAPE",loss2, " FC_loss_mape:",loss_FC)
            print("预测结果:",Pred_Result[0][0,15:20],"\n 目标: ",real_label[0,0,15:20])
np.save('data/Context_output.npy',output_data1)
np.save('data/GNN_output.npy',output_data2)
np.save('data/GNN_realdata.npy',real_data)
#np.save('data/GNN_context_matrix.npy',Context_matrix.squeeze(axis=0))

MSE: 7086.675   MAPE 31.098145  FC_loss_mape: 12.713832
预测结果: [ 93.763824 140.72855  155.22928   83.48794   97.77398 ] 
 目标:  [ 75.  79. 172.  74. 146.]
MSE: 42542.367   MAPE 39.518364  FC_loss_mape: 14.220515
预测结果: [137.90341 198.57657 211.73291 117.81196 137.03917] 
 目标:  [143. 117. 274. 118. 267.]
MSE: 32347.195   MAPE 36.78245  FC_loss_mape: 13.939212
预测结果: [133.1827  193.87022 195.4505  102.19683 128.1573 ] 
 目标:  [144. 107. 268. 113. 241.]
MSE: 2113.334   MAPE 48.84018  FC_loss_mape: 10.0731
预测结果: [ 95.75968  118.834785 154.40482  103.11043  152.98051 ] 
 目标:  [ 48.  49. 119.  47.  73.]
MSE: 76461.06   MAPE 30.125385  FC_loss_mape: 14.14144
预测结果: [ 92.68455 150.70296 150.1455   81.70289  97.44717] 
 目标:  [128. 113. 117.  49.  72.]
MSE: 38661.773   MAPE 38.105804  FC_loss_mape: 14.190813
预测结果: [ 90.0662   146.09795  147.25763   78.119415  93.577576] 
 目标:  [187. 151. 143.  68. 122.]
MSE: 37375.48   MAPE 30.20583  FC_loss_mape: 14.558494
预测结果: [117.10807  173.80698  163.77063   81.

NameError: name 'Context_matrix' is not defined

In [96]:
output_data2.shape, Pred_Result.shape

((4303, 108, 6), (1, 6, 108))

In [98]:
one_epoch_size*batch_size

4303

In [135]:
import numpy as np
SIP_E = np.load('ex_time_stamp_30min.npy')
SIP_30min_flow = np.load('Taxipick_sum_30m.npy')
SIP_sum = np.int32(np.sum(SIP_30min_flow, axis = 1))
SIP_sum_new = np.where(SIP_sum>1000, 1,0)
index = np.flatnonzero(SIP_sum_new)
SIP_30min_flow_new = np.zeros([index.shape[0],SIP_30min_flow.shape[1]])
for i, c in enumerate(list(index)):
    SIP_30min_flow_new[i,:]=SIP_30min_flow[c,:]

In [124]:
Adj_NYC = np.load('static_affinity.npy')
Adj_NYC.shape
new_adj = np.zeros([70,354])
new_adj_f = np.zeros([70,70]) 
for i, c in enumerate(list(index)):
    new_adj[i,:] = Adj_NYC[c,:]
for j, c in enumerate(list(index)):
    new_adj_f[:,j] = new_adj[:,c] 
SIP_30min_flow_new1 = np.zeros([70, 7248])
SIP_30min_flow_new1 = np.log(SIP_30min_flow_new+1)
np.save('NYC70_flow_30min_log.npy',SIP_30min_flow_new1)
np.save('NYC70_flow_30min.npy',SIP_30min_flow_new)
np.save('NYC70_Adj.npy',new_adj_f)

In [132]:
new_adj_f[12]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.73396765, 0.        , 0.75147227, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.84960787, 0.        , 0.        , 0.        , 0.6620576 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.86324318, 0.        ,
       0.        , 0.        , 0.        , 0.72947562, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.67672079, 0.        , 0.        , 0.        ])

In [134]:
SIP_30min_flow_new[0]

array([0., 0., 0., ..., 0., 0., 0.])

TypeError: only size-1 arrays can be converted to Python scalars

In [163]:
SIP_30min_flow_new1

array([[1.38629436, 0.69314718, 2.30258509, ..., 0.69314718, 0.69314718,
        0.        ],
       [1.60943791, 2.30258509, 2.63905733, ..., 0.69314718, 0.        ,
        0.69314718],
       [1.94591015, 2.89037176, 3.4657359 , ..., 1.09861229, 0.69314718,
        1.09861229],
       ...,
       [2.07944154, 2.19722458, 2.07944154, ..., 1.09861229, 1.09861229,
        1.09861229],
       [5.19295685, 4.44265126, 4.47733681, ..., 5.35185813, 5.29330482,
        5.40267738],
       [0.69314718, 1.38629436, 1.38629436, ..., 0.        , 1.09861229,
        0.69314718]])